In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from keras.preprocessing import image
from tqdm import tqdm

In [ ]:
target = 200
def load_data():
    df = pd.read_csv("../input/recognizance/train_label.csv")
    images = df['image'].values
    data = None

    for n,i in enumerate(images):
        if n<80:
            img = image.load_img("../input/recognizance/train/0/"+i,grayscale=True,target_size=(target,target))
        else:
            img = image.load_img("../input/recognizance/train/1/"+i,grayscale=True,target_size=(target,target))
        img = np.hstack(np.array(img))
        data = pd.concat([data,pd.DataFrame(img).transpose()])
        
    return df.join(data.reset_index(drop=True))


In [ ]:
df = load_data()

In [ ]:
def display_image(i,label):
    img = image.load_img("../input/recognizance/train/"+str(label)+"/"+i,grayscale=True,target_size=(target,target))
    img = np.array(img)
    plt.imshow(img,cmap='gray')
    

In [ ]:
display_image(df[['image','label']].iloc[1]['image'],df[['image','label']].iloc[1]['label'])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['image','label'],axis=1),df['label'], stratify=df['label'],test_size=0.3, random_state=42)

In [ ]:
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf','linear', 'poly']
             }  
  
grid = GridSearchCV(SVC(), param_grid,n_jobs=-1) 
  
grid.fit(X_train,y_train)

In [ ]:
pred = grid.predict(X_test)

In [ ]:
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))

In [ ]:
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf','linear', 'poly']
             }  
  
grid = GridSearchCV(SVC(), param_grid,n_jobs=-1) 
  
grid.fit(df.drop(['image','label'],axis=1),df['label'])

In [ ]:
grid.best_estimator_

In [ ]:
import os
files = []
for dirname, _, filenames in os.walk('/kaggle/input/recognizance/test/'):
    for filename in filenames:
        files.append(filename)

In [ ]:
pred = []
for i in files:
    img = image.load_img('../input/recognizance/test/'+i, target_size=(target,target),color_mode = "grayscale")
    img = np.hstack(np.array(img))
    pred.append(grid.predict(img.reshape(1,-1))[0])

In [ ]:
sub = pd.DataFrame(pred,files).reset_index()
sub.columns = ['image','label']
sub.to_csv('subgrid.csv',index=False)